<a href="https://colab.research.google.com/github/ghostwalkin/Multimodal-finetuning-ImgText-pair/blob/main/Finetune_CLIP_Amazon_products_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
from PIL import Image,UnidentifiedImageError
import concurrent.futures
from tqdm import tqdm

def chk_img(img):
  try:
    response = requests.get(img, stream=True, timeout=10) #added timeout.
    response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
    Image.open(response.raw).verify() #verify the image is not truncated.
    return None
  except (requests.exceptions.RequestException, OSError, UnidentifiedImageError) as e:
        return img

image_list=ds["train"]["imgUrl"]+ds["test"]["imgUrl"]+ds["valid"]["imgUrl"]
image_list=list(set(image_list))

with concurrent.futures.ThreadPoolExecutor() as executor:
    corrupt_image = list(tqdm(executor.map(chk_img, image_list), total=len(image_list), desc="Checking Images"))

corrupt_image=list(filter(None, corrupt_image))
print(corrupt_image)


Checking Images: 100%|██████████| 49755/49755 [09:56<00:00, 83.42it/s]

['https://m.media-amazon.com/images/I/51P8KSC3YwL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/71RY7664UdS._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/51mLU6F2PwL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/513UCzXgSOL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/51cZODeGAOS._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/61Kw7agWWkL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/51O8cPsHUdL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/71YLpR-uiiL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/418NHV1x-xL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/41zmR+k1LpL._MCnd_AC_UL320_.jpg']


In [2]:
from datasets import load_dataset,Dataset

In [3]:
ds=load_dataset("dparijat/amazon-image-title-triplet-250k")

# cleaning

In [8]:
len(corrupt_image)

10

In [4]:
corrupt_image=['https://m.media-amazon.com/images/I/51P8KSC3YwL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/71RY7664UdS._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/51mLU6F2PwL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/513UCzXgSOL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/51cZODeGAOS._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/61Kw7agWWkL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/51O8cPsHUdL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/71YLpR-uiiL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/418NHV1x-xL._AC_UL320_.jpg', 'https://m.media-amazon.com/images/I/41zmR+k1LpL._MCnd_AC_UL320_.jpg']

In [10]:
ds["train"]=ds["train"].filter(lambda x:x["imgUrl"] not in corrupt_image)
ds["test"]=ds["test"].filter(lambda x:x["imgUrl"] not in corrupt_image)
ds["valid"]=ds["valid"].filter(lambda x:x["imgUrl"] not in corrupt_image)
ds

Filter:   0%|          | 0/149975 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['imgUrl', 'title', 'negative'],
        num_rows: 149975
    })
    valid: Dataset({
        features: ['imgUrl', 'title', 'negative'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['imgUrl', 'title', 'negative'],
        num_rows: 49975
    })
})

In [11]:
from huggingface_hub import notebook_login
notebook_login()

# preprocessimg

In [13]:
import requests
from PIL import Image
def preprocess(batch):
  image_list=[Image.open(requests.get(url,stream=True).raw) for url in batch["imgUrl"]]
  batch["title"]=[x[:77] for x in batch["title"]]
  batch["negative"]=[x[:77] for x in batch["negative"]]

  return {
        "anchor": image_list,
        "positive": batch["title"],
        "negative": batch["negative"]
    }

columns_to_remove=["imgUrl","title"]

ds=ds.map(preprocess,remove_columns=columns_to_remove,batched=True)
ds["train"]=ds["train"].select_columns(["anchor","positive","negative"])
ds["test"]=ds["test"].select_columns(["anchor","positive","negative"])
ds["valid"]=ds["valid"].select_columns(["anchor","positive","negative"])

ds

Map:   0%|          | 0/149975 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/49975 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 149975
    })
    valid: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['anchor', 'positive', 'negative'],
        num_rows: 49975
    })
})

In [14]:
ds.push_to_hub("dparijat/amazon-image-title-triplet-250k-cleaned")


Uploading the dataset shards:   0%|          | 0/5 [00:00<?, ?it/s]

Map:   0%|          | 0/29995 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/300 [00:00<?, ?ba/s]

Map:   0%|          | 0/29995 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/300 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/24988 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/250 [00:00<?, ?ba/s]

Map:   0%|          | 0/24987 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/250 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/dparijat/amazon-image-title-triplet-250k-cleaned/commit/ba303ae7b59dd4945d8c3116367118bf0e4014f3', commit_message='Upload dataset', commit_description='', oid='ba303ae7b59dd4945d8c3116367118bf0e4014f3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/dparijat/amazon-image-title-triplet-250k-cleaned', endpoint='https://huggingface.co', repo_type='dataset', repo_id='dparijat/amazon-image-title-triplet-250k-cleaned'), pr_revision=None, pr_num=None)

In [ ]:
ds.cleanup_cache_files

# model download and freeze params


In [1]:
!pip install datasets sentence-transformers huggingface-hub --quiet

In [ ]:
ds=load_dataset("dparijat/amazon-image-title-triplet-250k-cleaned")

In [ ]:
from sentence_transformers import SentenceTransformer

model_name="sentence-transformers/clip-ViT-L-14"
model=SentenceTransformer(model_name)

In [ ]:
trainable_params=["projection"]
for name,param in model.named_parameters():
  if any(x in name for x in trainable_params):
    param.requires_grad=True
  else:
    param.requires_grad=False

In [ ]:
for name,param in model.named_parameters():
  if param.requires_grad:
    print(name,param,param.shape)

# evals

*   Tripleteval : check if (anchor,pos) is closer than (anchor,neg)
*   Recall@k: from the predictions w.r.t image is the embeddings for title is in top k predictions



In [ ]:
import torch
from torch import Tensor,nn
import torch.nn.functional as F

from sentence_transformers.evaluation import SentenceEvaluator


In [ ]:
#@title calulate recall@1 before fine-tuning

image_embeddings=model.encode(ds['train']["anchor"],batch_size=32,show_progress_bar=True,convert_to_tensor=True)
text_embeddings=model.encode(ds['train']["positive"],batch_size=32,show_progress_bar=True,convert_to_tensor=True)




In [ ]:
def calculate_recall_at_k(image_list:list, text_list: list, k=1):
    assert len(image_list) == len(text_list)
    image_embeddings = model.encode(image_list, batch_size=32, show_progress_bar=True, convert_to_tensor=True)
    text_embeddings = model.encode(text_list, batch_size=32, show_progress_bar=True, convert_to_tensor=True)

    correct = 0
    for i in range(len(image_list)):
        similarity = F.cosine_similarity(image_embeddings[i].unsqueeze(0), text_embeddings, dim=1)
        _, top_indices = torch.topk(similarity, k=k)
        if i in top_indices:
            correct += 1
    return correct / len(image_list)

recall_at_1 = calculate_recall_at_k(ds["train"]["anchor"], ds["train"]["positive"], k=1)
print(f"Recall@1: {recall_at_1}")

In [ ]:
recall_val=calculate_recall_at_k(ds["valid"]["anchor"], ds["valid"]["positive"], k=1)
print(f"Recall@1: {recall_val}")

In [ ]:
from typing import List, Dict
from sentence_transformers.evaluation import SentenceEvaluator
class ImageTextRetrievalEvaluator(SentenceEvaluator):
    def __init__(
        self,
        images: List,
        texts: List[str],
        name: str = '',
        k: int = 1,
        batch_size: int = 32,
        show_progress_bar: bool = False
    ):
        self.images = images
        self.texts = texts
        self.name = name
        self.k = k
        self.batch_size = batch_size
        self.show_progress_bar = show_progress_bar

    def __call__(self,
        model: SentenceTransformer,
        output_path: str = None,
        epoch: int = -1,
        steps: int = -1) -> Dict[str, float]:

        # Get embeddings for all images
        # Get embeddings for all images in batches
        img_embeddings=model.encode(self.images,
            batch_size=self.batch_size,
            show_progress_bar=self.show_progress_bar,
            convert_to_tensor=True

        )
        # Get embeddings for all texts in batches
        txt_embeddings=model.encode(self.texts,
            batch_size=self.batch_size,
            show_progress_bar=self.show_progress_bar,
            convert_to_tensor=True

        )


        correct = 0
        for i in range(img_embeddings.shape[0]):
            similarity = F.cosine_similarity(img_embeddings[i].unsqueeze(0), txt_embeddings, dim=1)
            _, top_indices = torch.topk(similarity, k=1)
            if i in top_indices:
                correct += 1

        recall_at_k=correct/img_embeddings.shape[0]


        return {f'{self.name}_Recall@{self.k}': recall_at_k}



def create_recall_evaluator(set_name, k=1):
    """
        Create triplet evaluator for "train", "valid", or "test" split
    """

    return ImageTextRetrievalEvaluator(
        images=ds[f"{set_name}"]["anchor"],
        texts=ds[f"{set_name}"]["positive"],
        name=f"clip_score-{set_name}",
        k=k
    )

In [ ]:
evaluator_recall_train = create_recall_evaluator("train", k=1)
evaluator_recall_valid = create_recall_evaluator("valid", k=1)


# training args and trainer

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.losses import MultipleNegativesRankingLoss,CachedMultipleNegativesRankingLoss

In [ ]:
#@title args

#loss=MultipleNegativesRankingLoss(model)
loss=CachedMultipleNegativesRankingLoss(model)

num_epochs = 10
batch_size = 32
lr = 1e-5
finetuned_model_name = "clip-amazon-product-title-similarity-v4"

train_args = SentenceTransformerTrainingArguments(
    output_dir=f"models/{finetuned_model_name}",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=lr,
    fp16=True,
    gradient_checkpointing=True,
    gradient_accumulation_steps=2,
    torch_empty_cache_steps=4,

    seed=42,
    lr_scheduler_type="linear",


    # Evaluation settings
    eval_strategy="epoch",
    eval_steps=1,
    logging_steps=1,
)

In [ ]:
#@title trainer
import os
os.environ["TORCH_LOGS"] = "+dynamo"  # Enable dynamo logs
os.environ["TORCHDYNAMO_VERBOSE"] = "1"

trainer = SentenceTransformerTrainer(
    model=model,
    args=train_args,
    train_dataset=ds["train"],
    eval_dataset=ds["valid"],
    evaluator=[evaluator_recall_train, evaluator_recall_valid],
    loss=loss,
    callbacks=None
)

trainer.train()

In [ ]:
model2=SentenceTransformer("/content/models/clip-amazon-product-title-similarity-v4/checkpoint-1870")

In [ ]:
model2.eval()

In [ ]:
def calculate_recall_at_kkk(sentencemodel,dataset_split, k=1):
    sentencemodel.eval()

    assert len(dataset_split["anchor"]) == len(dataset_split["positive"])
    image_embeddings = sentencemodel.encode(dataset_split["anchor"], batch_size=32, show_progress_bar=True, convert_to_tensor=True)
    text_embeddings = sentencemodel.encode(dataset_split["positive"], batch_size=32, show_progress_bar=True, convert_to_tensor=True)

    correct = 0
    for i in range(len(dataset_split["positive"])):
        similarity = F.cosine_similarity(image_embeddings[i].unsqueeze(0), text_embeddings, dim=1)
        _, top_indices = torch.topk(similarity, k=k)
        if i in top_indices:
            correct += 1
    return correct / len(dataset_split["positive"])

In [ ]:
calculate_recall_at_kkk(model2,ds["test"],k=1)

In [ ]:
calculate_recall_at_kkk(model,ds["test"],k=1)